# Input


## Embedding


In [24]:
import torch
import torch.nn as nn
import math
from torch.autograd import Variable

In [28]:
class Embeddings(nn.Module):

    def __init__(self,d_model,vocab):
        #d_model:dimension of embedding 
        #vocab:size of wordlist
        super(Embeddings,self).__init__()
        self.lut =nn.Embedding(vocab,d_model)
        self.d_model =d_model
    def forward(self,x):
        return self.lut(x)*math.sqrt(self.d_model)
    
d_model =512
vocab = 1000
x=torch.LongTensor([[1,2],[3,4]])
emb =Embeddings(d_model,vocab)
emb_rslt=emb(x)
print("embr",emb_rslt)
print("embrshape",emb_rslt.shape)

    

embr tensor([[[ -8.1050,  37.6993, -10.0451,  ..., -12.3506, -27.2399, -28.2641],
         [  9.6038,  18.9516,  13.7140,  ...,  -3.5545,  14.1081, -26.2569]],

        [[ 31.7261,  34.2334,  20.5876,  ..., -13.5841,  21.5117, -27.2552],
         [-37.2079,  18.6345,   7.0332,  ...,  14.6368,   7.4298,   1.4494]]],
       grad_fn=<MulBackward0>)
embrshape torch.Size([2, 2, 512])


In [ ]:
#nn.Embedding(vocab,d_model)
#integer in[0,vocab-1]->vector in R^d_model(randomly)
#integer in tensor lut place by place
embedding = nn.Embedding(10,3)
input = torch.LongTensor([[1,2,4,5],[2,3,4,9]])
print(embedding(input))
#10:max integer0-9
#3:each ->3d
#[[1,2,4,5],[2,3,4,9]]
#  v
#[ 1.8997,  0.6557, -0.1424]

tensor([[[ 1.8997,  0.6557, -0.1424],
         [-0.0872,  0.1228, -0.7261],
         [ 0.4782,  0.0221, -1.3902],
         [ 0.5069,  0.9979,  0.0901]],

        [[-0.0872,  0.1228, -0.7261],
         [ 1.5800,  0.5353,  0.6007],
         [ 0.4782,  0.0221, -1.3902],
         [ 0.1910, -1.2970, -0.9452]]], grad_fn=<EmbeddingBackward0>)


In [27]:
embedding = nn.Embedding(10,3,padding_idx=1)
input=torch.LongTensor([1,0])
print(embedding(input))
#[1,0]
# V
#[ 0.0000,  0.0000,  0.0000]
#padding_idx->0

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 1.6208,  0.2372, -1.3009]], grad_fn=<EmbeddingBackward0>)


## Position Encoding
